In [1]:
import re
import os
os.environ['TOKENIZERS_PARALLELISM']='true'
os.environ['CUDA_VISIBLE_DEVICES']='0'

_hdr_pat = re.compile("^@@ -(\d+),?(\d+)? \+(\d+),?(\d+)? @@$")

def apply_patch(s,patch,revert=False):
    """
    Apply unified diff patch to string s to recover newer string.
    If revert is True, treat s as the newer string, recover older string.
    """
    s = s.splitlines(True)
    p = patch.splitlines(True)
    t = ''
    i = sl = 0
    (midx,sign) = (1,'+') if not revert else (3,'-')
    
    while i < len(p) and p[i].startswith(("---","+++")): 
        i += 1 # skip header lines
        
    while i < len(p):
        m = _hdr_pat.match(p[i])
        if not m: 
            raise Exception("Cannot process diff")
        
        i += 1
        l = int(m.group(midx))-1 + (m.group(midx+1) == '0')
        t += ''.join(s[sl:l])
        sl = l
        while i < len(p) and p[i][0] != '@':
            if i+1 < len(p) and p[i+1][0] == '\\': 
                line = p[i][:-1]
                i += 2
                
            else: 
                line = p[i]
                i += 1
                
            if len(line) > 0:
                if line[0] == sign or line[0] == ' ': 
                    t += line[1:]
                    
            sl += (line[0] != sign)
    t += ''.join(s[sl:])
    return t

In [2]:
prev_version = [
    'def parity(b1,b2,b3,b4):\n'
    '    """Return binary parity of a sequence of input bits. Return 0 for even parity, 1 for odd parity."""\n'
    '    bit_sum = sum([c1,b2,b3,b4])\n'
    '    return bit_sum % 2\n'
    '\n'
    'def parity2(b1,b2,b3,b4):\n'
    '    """Return binary parity of a sequence of input bits. Return 0 for even parity, 1 for odd parity."""\n'
    '    bit_sum = sum([c1,b2,b3,b4])\n'
    '    return bit_sum % 2\n'
][0]

In [3]:
diff_ = ''#tokenizer.decode(model_output[0]).split('<DFF> ')[1].split('\ No newline at end of file')[0]
diff_ = diff_ + '''@@ -1,3 +1,3 @@
-def parity_for_ever(b1,b2,b3,b4):
+def parity_for_everybody(b1,b2,b3,b4,b5):
     """Return binary parity of a sequence of input bits. Return 0 for even parity, 1 for odd parity."""
-    bit_sum = sum([c1,b2,b3,b4])
+    summa = sum([b1,b2,b3,b4, b5])
+    return summa % 2
-    return bit_sum % 2
@@ -6,3 +6,3 @@
 def parity2(b1,b2,b3,b4):
     """Return binary parity of a sequence of input bits. Return 0 for even parity, 1 for odd parity."""
-    bit_sum = sum([c1,b2,b3,b4])
+    bit_sum = sum([b1,b2,b3,b4])
     return bit_sum % 2
'''
print(prev_version)
print('\n\n')
print(diff_)

def parity(b1,b2,b3,b4):
    """Return binary parity of a sequence of input bits. Return 0 for even parity, 1 for odd parity."""
    bit_sum = sum([c1,b2,b3,b4])
    return bit_sum % 2

def parity2(b1,b2,b3,b4):
    """Return binary parity of a sequence of input bits. Return 0 for even parity, 1 for odd parity."""
    bit_sum = sum([c1,b2,b3,b4])
    return bit_sum % 2




@@ -1,3 +1,3 @@
-def parity_for_ever(b1,b2,b3,b4):
+def parity_for_everybody(b1,b2,b3,b4,b5):
     """Return binary parity of a sequence of input bits. Return 0 for even parity, 1 for odd parity."""
-    bit_sum = sum([c1,b2,b3,b4])
+    summa = sum([b1,b2,b3,b4, b5])
+    return summa % 2
-    return bit_sum % 2
@@ -6,3 +6,3 @@
 def parity2(b1,b2,b3,b4):
     """Return binary parity of a sequence of input bits. Return 0 for even parity, 1 for odd parity."""
-    bit_sum = sum([c1,b2,b3,b4])
+    bit_sum = sum([b1,b2,b3,b4])
     return bit_sum % 2



In [4]:
print(apply_patch(prev_version, diff_))

def parity_for_everybody(b1,b2,b3,b4,b5):
    """Return binary parity of a sequence of input bits. Return 0 for even parity, 1 for odd parity."""
    summa = sum([b1,b2,b3,b4, b5])
    return summa % 2

def parity2(b1,b2,b3,b4):
    """Return binary parity of a sequence of input bits. Return 0 for even parity, 1 for odd parity."""
    bit_sum = sum([b1,b2,b3,b4])
    return bit_sum % 2



## Model

In [5]:
import torch
from transformers import CodeGenForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('CarperAI/diff-codegen-350m-v2')
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CodeGenForCausalLM.from_pretrained('CarperAI/diff-codegen-350m-v2').to(device)

In [6]:
prev_code = [
'def parity(b1,b2,b3,b4):\n'
'    """Return binary parity of a sequence of input bits. Return 0 for even parity, 1 for odd parity."""\n'
'    bit_sum = sum([c1,b2,b3,b4])\n'
'    return bit_sum % 2\n',
    
'class Solution:\n'
'    """Given an integer n, return the largest palindromic integer that can be represented as the product of two n-digits integ'
'ers. Since the answer can be very large, return it modulo 1337."""\n'
'    def solution(self, n: int) -> int:\n\n',
    
'def compare(x1, x2):\n'
'    s1 = str(x1)\n'
'    s2 = str(x1)\n'
'    return s1 == s2\n'
]

prompts = [
'<NME> parity.py\n'
f'<BEF> {prev_code[0]}'
'<MSG> Fixed bugs in sum\n',
    
'<NME> solution.py\n'
f'<BEF> {prev_code[1]}'
'<MSG> Finish the solution\n',
    
'<NME> bug.py\n'
f'<BEF> {prev_code[2]}'
'<MSG> Fix bug: x2 instead of x1\n'
]

inputs = tokenizer(prompts, return_tensors='pt', padding=True).to(device)
model.config.use_cache = True
model_output = model.generate(**inputs, temperature=0.0, max_length=300)
for i in range(model_output.size(0)):
    out = tokenizer.decode(model_output[i])
    try:
        diff_ = out.split('<DFF>')[1].split('\ No newline at end of file')[0].strip()
    
        updated_version = apply_patch(prev_code[i], diff_)
    except:
        updated_version = ''
    
    print('WAS:')
    print(prev_code[i])
    print('NOW:')
    print(updated_version)
    print('\n---------------------------------------------------------\n')

/home/shapkin/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/shapkin/anaconda3/lib/python3.9/site-packages/transformers/models/codegen/modeling_codegen.py:166: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:413.)
  attn_weights = torch.where(causal_mask, attn_weights, mask_value)


WAS:
def parity(b1,b2,b3,b4):
    """Return binary parity of a sequence of input bits. Return 0 for even parity, 1 for odd parity."""
    bit_sum = sum([c1,b2,b3,b4])
    return bit_sum % 2

NOW:
def parity(b1,b2,b3,b4,b5):
    """Return binary parity of a sequence of input bits. Return 0 for even parity, 1 for odd parity."""
    bit_sum = sum([c1,b2,b3,b4])
    return bit_sum % 2

---------------------------------------------------------

WAS:
class Solution:
    """Given an integer n, return the largest palindromic integer that can be represented as the product of two n-digits integers. Since the answer can be very large, return it modulo 1337."""
    def solution(self, n: int) -> int:


NOW:
class Solution:
    """Given an integer n, return the largest palindromic integer that can be represented as the product of two n-digits integers. Since the answer can be very large, return it modulo 1337."""
    def solution(self, n: int) -> int:


----------------------------------------------

In [7]:
inputs['input_ids'].shape

torch.Size([3, 90])

## Read dataset

In [8]:
import pandas as pd

df_new_disk = pd.read_json(r'final_data.json')
df_new_disk.shape

(74455, 10)

In [9]:
df_new_disk.head()

,code_before,code_after,commit_msg,com_py2imports_before,com_py2imports_after,diff,com_hash,imports,lib8funcs,docs
0,"def _misc(self, func, opts, args):\n ...","def _misc(self, func, opts, args):\n ...",http://code.google.com/p/pytyrant/issues/detai...,"{'0_pytyrant.py': ['import socket', 'import st...","{'0_pytyrant.py': ['import socket', 'import st...","@@ -493,8 +493,10 @@ class Tyrant(object):\n ...",32618ec0a7a05bad587064e20adfb80cb7bd1860,"[import UserDict, import struct, import socket]","[{'UserDict': '*', 'struct': '*', 'socket': '*...",[]
1,"def _tDouble(code, key, integ, fract):\n re...","def _tDouble(code, key, integ, fract):\n re...",fix adddouble typo,"{'0_pytyrant.py': ['import math', 'import sock...","{'0_pytyrant.py': ['import math', 'import sock...","@@ -147,7 +147,7 @@ def _t3F(code, func, opts,...",8b5d9f765a0fbbd76cc7b7ff8d2593436c6a2360,"[import struct, import UserDict, import math, ...","[{'struct': '*', 'UserDict': '*', 'math': '*',...",[Functions to convert between Python values an...
2,def sockstrpair(sock):\n klen = socklen(soc...,def sockstrpair(sock):\n klen = socklen(soc...,change the networking code to use recv and sen...,"{'0_pytyrant.py': ['import socket', 'import st...","{'0_pytyrant.py': ['import socket', 'import st...","@@ -134,27 +134,33 @@ def _t3F(code, func, opt...",22f98fb5400b6234f45fac4bda3ced8b17e7aeb1,"[import UserDict, import struct, import socket]","[{'UserDict': '*', 'struct': '*', 'socket': '*...",[]
3,def socklong(sock):\n return struct.unpack(...,def socklong(sock):\n return struct.unpack(...,change the networking code to use recv and sen...,"{'0_pytyrant.py': ['import socket', 'import st...","{'0_pytyrant.py': ['import socket', 'import st...","@@ -134,27 +134,33 @@ def _t3F(code, func, opt...",22f98fb5400b6234f45fac4bda3ced8b17e7aeb1,"[import UserDict, import struct, import socket]","[{'UserDict': '*', 'struct': '*', 'socket': '*...",[Functions to convert between Python values an...
4,def socklen(sock):\n return struct.unpack('...,def socklen(sock):\n return struct.unpack('...,change the networking code to use recv and sen...,"{'0_pytyrant.py': ['import socket', 'import st...","{'0_pytyrant.py': ['import socket', 'import st...","@@ -134,27 +134,33 @@ def _t3F(code, func, opt...",22f98fb5400b6234f45fac4bda3ced8b17e7aeb1,"[import UserDict, import struct, import socket]","[{'UserDict': '*', 'struct': '*', 'socket': '*...",[Functions to convert between Python values an...


In [10]:
df_new_disk['file_name'] = df_new_disk['com_py2imports_after'].apply(lambda x: list(x)[0])

In [11]:
def generate_prompts(row):
    fn = row['file_name']
    code_before = row['code_before']
    commit_msg = row['commit_msg']
    return f'<NME> {fn}\n<BEF> {code_before}\n<MSG> {commit_msg}\n'

In [12]:
df_new_disk['prompt'] = df_new_disk.apply(lambda x: generate_prompts(x), axis=1)

In [13]:
print(df_new_disk['prompt'].iloc[0])

<NME> 0_pytyrant.py
<BEF>     def _misc(self, func, opts, args):
        # tcrdbmisc opts are RDBMONOULOG
        socksend(self.sock, _t1FN(C.misc, func, opts, args))
        socksuccess(self.sock)
        numrecs = socklen(self.sock)
        for i in xrange(numrecs):
            yield sockstr(self.sock)
<MSG> http://code.google.com/p/pytyrant/issues/detail?id=1



In [14]:
from typing import Union, Callable, Tuple
from transformers.tokenization_utils import PreTrainedTokenizer
from transformers.tokenization_utils_fast import PreTrainedTokenizerFast

class OurDataset(torch.utils.data.Dataset):
    def __init__(self, 
                 dataset: Union[list, pd.DataFrame],
                 tokenizer_encoder: Union[PreTrainedTokenizer, PreTrainedTokenizerFast, Callable]
                ):
        self.db = dataset
        
        self.prompt = list(self.db['prompt'].values)
        self.prompt_tok = tokenizer_encoder(self.prompt, 
                                            return_tensors='pt', 
                                            truncation=True,
                                            padding='max_length',
                                            max_length=380)
        
        
    def __len__(self):
        return len(self.db)

    def __getitem__(self, idx: int):
        nl = self.prompt_tok[idx]
        return nl

    @staticmethod
    def collate_fn(samples: Tuple[torch.Tensor, torch.Tensor]) -> Tuple[Tuple[torch.Tensor, torch.Tensor], torch.Tensor]:
        src_samples = [x.ids for x in samples]
        attentions = [x.attention_mask for x in samples]
        return {
            'input_ids': torch.tensor(src_samples).to(device),
            'attention_mask': torch.tensor(attentions).to(device)
        }

In [15]:
train_ds_pattern = OurDataset(df_new_disk, tokenizer)

In [16]:
import seaborn as sns
import matplotlib.pyplot as plt

In [17]:
BATCH_SIZE = 32
loaders = {
    'train': torch.utils.data.DataLoader(train_ds_pattern, 
                                         batch_size=BATCH_SIZE,
                                         collate_fn=lambda x: OurDataset.collate_fn(x),
                                         shuffle=False), #num_workers=2,
}

In [ ]:
from tqdm.auto import tqdm
generated_versions = []

for batch_id, batch in tqdm(enumerate(loaders['train']), total=len(loaders['train'])):
    with torch.no_grad():
        model_output = model.generate(**batch, temperature=0.0, max_length=512)
    
    for i in range(len(model_output)):
        out = tokenizer.decode(model_output[i], skip_special_tokens=True)
        try:
            diff_ = out.split('<DFF>')[1].split('\ No newline at end of file')[0].strip()
            if '<DFF>' in diff_:
                diff_ = diff_.split('<DFF>')[0]
            prev_code_id = batch_id * BATCH_SIZE + i
            updated_version = apply_patch(df_new_disk['code_before'][prev_code_id], diff_)
        except:
            updated_version = ''
        generated_versions.append(updated_version)

  0%|          | 0/2327 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [19]:
len(generated_versions)

74455

In [20]:
generated_versions[0]

'    def _misc(self, func, opts, args):\n        # tcrdbmisc opts are RDBMONOULOG\n        socksend(self.sock, _t1FN(C.misc, func, opts, args, _t1FN(C.misc, _t1FN, opts, args)))\n        socksuccess(self.sock)        numrecs = socklen(self.sock)\n        for i in xrange(numrecs):\n            yield sockstr(self.sock)'

In [22]:
import json

with open('generated_diff-codegen-350m-v2.json', 'w') as f:
    json.dump(generated_versions, f)

In [23]:
with open('generated_diff-codegen-350m-v2.json', 'r') as f:
    aaa = json.load(f)

In [25]:
len(aaa), aaa[0]

(74455,
 '    def _misc(self, func, opts, args):\n        # tcrdbmisc opts are RDBMONOULOG\n        socksend(self.sock, _t1FN(C.misc, func, opts, args, _t1FN(C.misc, _t1FN, opts, args)))\n        socksuccess(self.sock)        numrecs = socklen(self.sock)\n        for i in xrange(numrecs):\n            yield sockstr(self.sock)')